# Gamma-Gamma 模型 - 预测客户消费金额
> By SherryAGI | 结合 BG/NBD 计算完整 CLV

## Gamma-Gamma 模型简介

BG/NBD 预测**购买次数**，Gamma-Gamma 预测**每次消费金额**。

两者结合 = 完整的 CLV 预测

```
CLV = 预测购买次数 × 预测平均消费金额 × 毛利率
```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.utils import summary_data_from_transaction_data

print('🚀 Gamma-Gamma 消费预测模型 by SherryAGI')

## 1. 准备数据

In [ ]:
# 模拟交易数据
np.random.seed(42)
n_transactions = 10000

transactions = pd.DataFrame({
    'customer_id': np.random.randint(1, 1001, n_transactions),
    'date': pd.to_datetime('2024-01-01') + pd.to_timedelta(np.random.randint(0, 365, n_transactions), unit='D'),
    'amount': np.random.exponential(100, n_transactions)
})

# 转换为 RFM 格式
rfm = summary_data_from_transaction_data(
    transactions,
    customer_id_col='customer_id',
    datetime_col='date',
    monetary_value_col='amount'
)

# Gamma-Gamma 需要 frequency > 0
rfm_gg = rfm[rfm['frequency'] > 0]
print(f'有重复购买的客户: {len(rfm_gg)}')

## 2. 训练模型

In [ ]:
# 训练 BG/NBD 模型
bgf = BetaGeoFitter(penalizer_coef=0.0)
bgf.fit(rfm['frequency'], rfm['recency'], rfm['T'])

# 训练 Gamma-Gamma 模型
ggf = GammaGammaFitter(penalizer_coef=0.0)
ggf.fit(rfm_gg['frequency'], rfm_gg['monetary_value'])

print('Gamma-Gamma 模型参数:')
print(ggf.summary)

## 3. 预测平均消费金额

In [ ]:
# 预测每个客户的平均消费金额
rfm_gg['predicted_monetary'] = ggf.conditional_expected_average_profit(
    rfm_gg['frequency'],
    rfm_gg['monetary_value']
)

# 对比实际 vs 预测
print(f'实际平均消费: ¥{rfm_gg["monetary_value"].mean():.2f}')
print(f'预测平均消费: ¥{rfm_gg["predicted_monetary"].mean():.2f}')

## 4. 计算完整 CLV

In [ ]:
# 计算 12 个月的 CLV
time_months = 12
discount_rate = 0.01  # 月折现率
gross_margin = 0.3    # 毛利率 30%

rfm_gg['CLV'] = ggf.customer_lifetime_value(
    bgf,
    rfm_gg['frequency'],
    rfm_gg['recency'],
    rfm_gg['T'],
    rfm_gg['monetary_value'],
    time=time_months,
    discount_rate=discount_rate
) * gross_margin

print(f'\n📊 CLV 统计 (未来 {time_months} 个月):')
print(rfm_gg['CLV'].describe())

## 5. CLV 排名

In [ ]:
# Top 10 高价值客户
top_customers = rfm_gg.nlargest(10, 'CLV')[['frequency', 'monetary_value', 'predicted_monetary', 'CLV']]
top_customers['CLV'] = top_customers['CLV'].apply(lambda x: f'¥{x:.2f}')

print('🏆 Top 10 高价值客户:')
top_customers

## 6. 可视化

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# CLV 分布
rfm_gg['CLV'].hist(bins=50, ax=axes[0], color='steelblue', edgecolor='white')
axes[0].set_title('CLV 分布 by SherryAGI')
axes[0].set_xlabel('CLV (¥)')
axes[0].axvline(rfm_gg['CLV'].median(), color='red', linestyle='--', label=f'中位数: ¥{rfm_gg["CLV"].median():.0f}')
axes[0].legend()

# 实际 vs 预测消费
axes[1].scatter(rfm_gg['monetary_value'], rfm_gg['predicted_monetary'], alpha=0.5)
axes[1].plot([0, rfm_gg['monetary_value'].max()], [0, rfm_gg['monetary_value'].max()], 'r--', label='完美预测')
axes[1].set_title('实际 vs 预测消费金额')
axes[1].set_xlabel('实际平均消费 (¥)')
axes[1].set_ylabel('预测平均消费 (¥)')
axes[1].legend()

plt.tight_layout()
plt.show()

## 7. 业务决策

| CLV 分层 | 客户数 | 策略 |
|----------|--------|------|
| 💎 Top 10% | ~100 | VIP 专属服务 |
| 🥇 Top 25% | ~250 | 会员升级激励 |
| 📊 中间 50% | ~500 | 常规营销 |
| ⚠️ 底部 25% | ~250 | 低成本维护 |

In [ ]:
# 计算获客成本上限
avg_clv = rfm_gg['CLV'].mean()
print(f'\n💰 业务指标:')
print(f'平均 CLV: ¥{avg_clv:.2f}')
print(f'建议 CAC 上限: ¥{avg_clv/3:.2f} (CLV:CAC = 3:1)')
print(f'总客户价值: ¥{rfm_gg["CLV"].sum():,.0f}')

---
**By SherryAGI** | [DigitalTransformationAI](https://github.com/AIB612/DigitalTransformationAI)